# 합성곱 신경망 

* ConvNet의 정의
* ConvNet이 컴퓨터 비전 관련 작업에 잘 맞는 이유


In [0]:
### 간단한 ConvNet 만들기


from keras import layers
from keras import models

model = models.Sequential()

# MNIST image format: (heigth, width, channels) = ( 28, 28, 1(grayscale) )
# input_shape = (image_height, image_width, image_channels)

# Conv2D(output_depth, (window_height, window_width))
# output_depth: 합성곱으로 계산할 필터의 수
# (window_height, window_width): 입력으로부터 뽑아낼 패치의 크기. 주로 3x3 또는 5x5 사용.

# 최대 풀링(max pooling): 보통 2x2 윈도우와 스트라이드(패치 이동 간격) 1을 이용해 특성 맵을 절반 크기로 다운샘플링한다.
# 입력 특성 맵에서 윈도우에 맞는 패치를 추출하고, 각 채널별로 최댓값을 출력하는 것이다.
# 처리할 특성 맵의 가중치 개수를 줄이기 위한 다운샘플링 방법으로 많이 사용된다.
# 최대 풀링 방법은 단순히 윈도우를 듬성듬성하게 슬라이딩하거나 평균을 취하는 것이 아니라 채널별 최댓값을 출력하므로, 다른 방법들보다 다운샘플링에 효과적이다.

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Using TensorFlow backend.


In [0]:
# Conv2D와 MaxPooling2D 층의 출력은, (height, width, channels) 크기의 3D 텐서이다.
# 높이와 너비 차원은 네트워크가 깊어질수록 작아지는 경향이 있다.
# 채널의 수는, Conv2D 층에 전달된 첫 번째 매개변수에 의해 조절된다. (32개 또는 64개)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [0]:
### ConvNet 위에 분류기 추가하기


# 마지막 층의 출력 텐서(3, 3, 64)를 완전 연결 네트워크(Dense 층을 쌓은 분류기)에 주입한다.
# 이 분류기는 1D 벡터를 처리하므로, 3D 출력을 1D 텐서로 펼쳐야 한다.
# 그 다음, 몇 개의 Dense 층을 추가한다.

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
# 10개의 클래스(숫자 0~9)를 분류하기 위해, 마지막 층의 출력 크기를 10으로 하고, softmax 활성화 함수를 사용한다.
model.add(layers.Dense(10, activation='softmax'))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [0]:
### MNIST 이미지에 ConvNet 훈련하기


from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))  # 6만 개의 train data
train_images = train_images.astype('float32') / 255  # [0, 255] 사이의 값을 0과 1 사이의 값으로~

test_images = test_images.reshape((10000, 28, 28, 1))  # 1만 개의 test data
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)  # 레이블을 범주형으로 인코딩 (one-hot encoding)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)


11493376/11490434 [==============================] - 0s 0us/step


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/5
60000/60000 [==============================] - 53s 889us/step - loss: 0.2152 - acc: 0.9338
Epoch 2/5
60000/60000 [==============================] - 52s 875us/step - loss: 0.0619 - acc: 0.9815
Epoch 3/5
60000/60000 [==============================] - 53s 876us/step - loss: 0.0437 - acc: 0.9868
Epoch 4/5
60000/60000 [==============================] - 52s 863us/step - loss: 0.0348 - acc: 0.9904
Epoch 5/5
60000/60000 [==============================] - 52s 866us/step - loss: 0.0300 - acc: 0.9920


In [0]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

10000/10000 [==============================] - 3s 267us/step
0.9879


## 완전 연결 층(Dense 층)과 합성곱 층의 근본적인 차이
* Dense 층은 입력 특성 공간에 있는 전역 패턴(ex. 숫자 이미지의 모든 픽셀에 걸친 패턴)을 학습한다.
* 그러나, 합성곱 층은 지역 패턴을 학습한다.
  * 이미지일 경우, 작은 2D window로 입력에서 패턴을 찾는다.
  * 이 예제에서 윈도우는 모두 3x3 크기
  
  
## ConvNet의 성질
1. 학습된 패턴은 평행 이동 불변성(translation invariant)을 가진다.
  * convNet이 이미지의 특정 위치에서 학습한 패턴은 이미지의 다른 위치에서도 인식될 수 있다.
  * **효율적인 이미지 처리가 가능**
  
2. ConvNet은 패턴의 공간적 계층 구조를 학습할 수 있다.
  * 첫 번째 ConvNet 층이 edge같은 작은 지역 패턴을 학습하고,
  * 두 번째 ConvNet 층은 첫 번째 층의 특성으로 구성된 더 큰 패턴을 학습하고, 
  * ...
  * **매우 복잡하고 추상적인 시각적 개념을 효과적으로 학습 가능**

## 합성곱 연산의 원리

* 합성곱 연산은 **특성 맵(feature map)**이라는 3D 텐서에 적용된다.
  * (높이, 너비, 깊이)
  * RGB 이미지 -> 3개의 컬러 채널 -> 깊이=3(차원)
  * 흑백 이미지 -> 깊이=1(차원)
  
* 합성곱 연산은 
  * 입력 특성 맵에서 작은 패치(patch)들을 추출하고, 
    * ex. 입력 특성 맵 (28, 28, 1)
  * 모든 패치에 각각 같은 변환을 적용하여 
  * **출력 특성 맵(output feature map)**을 만든다.
    * ex. 출력 특성 맵 (26, 26, 32)
  
* 출력 특성 맵도 (높이, 너비, 깊이)의 3D 텐서이다.
  * 그러나, 출력 텐서의 깊이는 층의 매개변수에 의해 결정되므로, 상황에 따라 다르다.
  * **출력 텐서의 깊이**는 RGB 입력과 같은 특정 컬러가 아니라, 일종의 **필터(filter)**를 의미한다.
    * ex. 입력에 대해 32개의 필터를 적용
    * 32개의 필터 각각은, 입력에 대한 필터의 응답 맵(response map)으로 26x26 크기의 배열 값을 가진다.
    * 이는, 입력의 각 위치에서 필터 패턴에 대한 응답을 나타낸다.

* 필터는 입력 데이터의 어떤 특성을 인코딩한다.
  * ex. 하나의 필터가 '입력에 얼굴이 있는지'를 인코딩
 